In [1]:
import pandas as pd
import numpy as np
import pickle

df1 = pd.read_csv("../files/input/train_data.csv.zip", index_col=False, compression="zip")
df1_ = df1.copy()
df1_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          21000 non-null  int64
 1   LIMIT_BAL                   21000 non-null  int64
 2   SEX                         21000 non-null  int64
 3   EDUCATION                   21000 non-null  int64
 4   MARRIAGE                    21000 non-null  int64
 5   AGE                         21000 non-null  int64
 6   PAY_0                       21000 non-null  int64
 7   PAY_2                       21000 non-null  int64
 8   PAY_3                       21000 non-null  int64
 9   PAY_4                       21000 non-null  int64
 10  PAY_5                       21000 non-null  int64
 11  PAY_6                       21000 non-null  int64
 12  BILL_AMT1                   21000 non-null  int64
 13  BILL_AMT2                   21000 non-null  int64
 14  BILL_A

In [2]:
df2 = pd.read_csv("../files/input/test_data.csv.zip", index_col=False, compression="zip")
df2_=df2.copy()
df2_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          9000 non-null   int64
 1   LIMIT_BAL                   9000 non-null   int64
 2   SEX                         9000 non-null   int64
 3   EDUCATION                   9000 non-null   int64
 4   MARRIAGE                    9000 non-null   int64
 5   AGE                         9000 non-null   int64
 6   PAY_0                       9000 non-null   int64
 7   PAY_2                       9000 non-null   int64
 8   PAY_3                       9000 non-null   int64
 9   PAY_4                       9000 non-null   int64
 10  PAY_5                       9000 non-null   int64
 11  PAY_6                       9000 non-null   int64
 12  BILL_AMT1                   9000 non-null   int64
 13  BILL_AMT2                   9000 non-null   int64
 14  BILL_AMT

In [3]:
# def preprocess_data(df):

#     df = df.rename(columns={'default payment next month': 'default'})
#     df = df.drop(columns=['ID'])
#     df = df.dropna() 
#     df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

#     #df['EDUCATION'] = df['EDUCATION'].apply(lambda x: x if x > 4 else x)
#     #df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 0 if int(x) > 4 else int(x))
#     df['EDUCATION'] = df['EDUCATION'].fillna(0)
#     df = df.loc[df['EDUCATION'] != 0]
#     df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 'others' if x > 4 else str(x))

    
#     df = df.loc[df['MARRIAGE'] != 0]
#     df['EDUCATION'] = df['EDUCATION'].astype(str)
#     df['EDUCATION'] = df['EDUCATION'].astype('category')
#     df['MARRIAGE'] = df['MARRIAGE'].astype('category')
#     df['SEX'] = df['SEX'].astype('category')
    

#     return df
# df1_ = preprocess_data(df1)
# df2_ = preprocess_data(df2)
# df2_['EDUCATION'].unique()

In [4]:
def preprocess_data(df):
    # Renombrar columnas y eliminar irrelevantes
    df = df.rename(columns={'default payment next month': 'default'})
    df = df.drop(columns=['ID'])
    
    # Transformar y filtrar EDUCATION
    df['EDUCATION'] = df['EDUCATION'].apply(lambda x: '4' if x > 4 else str(x))
    df = df[df['EDUCATION'] != '0']
    #df = df[df['EDUCATION'] != 'others']
    df['EDUCATION'] = df['EDUCATION'].astype(int)
    df['EDUCATION'] = df['EDUCATION'].astype('category')

    # Filtrar valores inválidos de MARRIAGE
    df = df.loc[df['MARRIAGE'] != 0]
 
    # Convertir columnas categóricas
    df['MARRIAGE'] = df['MARRIAGE'].astype('category')
    df['SEX'] = df['SEX'].astype('category')

    return df

df1_ = preprocess_data(df1)
df2_ = preprocess_data(df2)
df1_['EDUCATION'].unique()

[3, 2, 1, 4]
Categories (4, int64): [1, 2, 3, 4]

In [5]:
# Contar valores faltantes por columna
faltantes_por_columna = df1_.isnull().sum()

print("\nValores faltantes por columna:")
print(faltantes_por_columna)



Valores faltantes por columna:
LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64


In [6]:
def make_train_test_split(train_data, test_data, target_column):

    # Separar características (X) y etiquetas (y) del conjunto de entrenamiento
    X_train = train_data.drop(columns=[target_column])
    y_train = train_data[target_column]
    
    # Separar características (X) y etiquetas (y) del conjunto de prueba
    X_test = test_data.drop(columns=[target_column])
    y_test = test_data[target_column]
    
    return X_train, X_test, y_train, y_test

# Llamar a la función con tus datos
X_train, X_test, y_train, y_test = make_train_test_split(
    train_data=df1_,
    test_data=df2_,
    target_column='default'
)

In [7]:
def make_pipeline(estimator):

    from sklearn.compose import ColumnTransformer
    from sklearn.feature_selection import SelectKBest, f_classif
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    
    categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']
    
    transformer = ColumnTransformer(
        transformers=[
           ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
          
        ],
        remainder='passthrough',
    )

    selectkbest = SelectKBest(score_func=f_classif)

    pipeline = Pipeline(
        steps=[
            ("tranformer", transformer),
            ("selectkbest", selectkbest),
            ("estimator", estimator),
        ],
        verbose=False,
    )

    return pipeline

In [8]:
known_values = ['1', '2', '3']
data_values = df2_['EDUCATION'].unique()

# Verificar diferencias
diff = set(data_values) - set(known_values)
print("Valores no reconocidos:", diff)


Valores no reconocidos: {1, 2, 3, 4}


In [9]:
def make_grid_search(estimator, param_grid, cv=10):

    from sklearn.model_selection import GridSearchCV

    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=cv,
        scoring='balanced_accuracy',
        n_jobs=-1
    )

    return grid_search

In [10]:
def save_estimator(estimator):
    import os
    import gzip
    import pickle

    # Crear la carpeta si no existe
    os.makedirs("../files/models", exist_ok=True)

    # Guardar el modelo con compresión gzip
    with gzip.open("../files/models/model.pkl.gz", "wb") as file:
        pickle.dump(estimator, file)


In [11]:
def load_estimator():
    import os
    import gzip
    import pickle

    # Verificar si el archivo existe
    if not os.path.exists("../files/models/model.pkl.gz"):
        return None

    # Cargar el modelo desde un archivo comprimido
    with gzip.open("../files/models/model.pkl.gz", "rb") as file:
        estimator = pickle.load(file)

    return estimator


In [12]:
def train_RandomForestClassifier():

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import mean_absolute_error

    # data, target = load_data()

    X_train, X_test, y_train, y_test = make_train_test_split(
            train_data=df1_,
            test_data=df2_,
            target_column='default'
    )

    pipeline = make_pipeline(
        estimator=RandomForestClassifier(random_state=42),
    )

    estimator = make_grid_search(
        estimator=pipeline,
        param_grid={
                'estimator__n_estimators': [500],
                'estimator__max_depth': [100],
                'estimator__min_samples_split': [2],
                'estimator__min_samples_leaf': [2],
                
        },
        cv=10,
    )

    estimator.fit(X_train, y_train)
    print(estimator.fit(X_train, y_train))
    best_estimator = load_estimator()
    
    if best_estimator is not None:

        saved_mae = mean_absolute_error(
            y_true=y_test, y_pred=best_estimator.predict(X_test)
        )

        current_mae = mean_absolute_error(
            y_true=y_test, y_pred=estimator.predict(X_test)
        )

        if saved_mae < current_mae:
            estimator = best_estimator

    save_estimator(estimator)


train_RandomForestClassifier()

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tranformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('selectkbest', SelectKBest()),
                                       ('estimator',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'estimator__max_depth': [100],
                         'estimator__min_samples_leaf': [2],
                         'estimator__min_s

In [13]:
import os
import json
from sklearn.metrics import (
    classification_report, precision_score, recall_score, f1_score,
    balanced_accuracy_score, confusion_matrix
)

def save_metrics_and_confusion_matrices(estimator, X_train, y_train, X_test, y_test):
    """
    Genera las métricas y matrices de confusión y las guarda en 'files/output/metrics.json'.
    """
    if estimator is None:
        raise ValueError("El modelo no se pudo cargar. Asegúrate de haber guardado el modelo correctamente.")
    
    # Predicciones
    y_train_pred = estimator.predict(X_train)
    y_test_pred = estimator.predict(X_test)

    # Calcular métricas con promedio ponderado
    metrics_train = {
        "type": "metrics",
        "dataset": "train",
        "precision": precision_score(y_train, y_train_pred, average='binary'),
        "balanced_accuracy": balanced_accuracy_score(y_train, y_train_pred),
        "recall": recall_score(y_train, y_train_pred, average='binary'),
        "f1_score": f1_score(y_train, y_train_pred, average='binary')
    }

    metrics_test = {
        "type": "metrics",
        "dataset": "test",
        "precision": precision_score(y_test, y_test_pred, average='binary'),
        "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
        "recall": recall_score(y_test, y_test_pred, average='binary'),
        "f1_score": f1_score(y_test, y_test_pred, average='binary')
    }

    # Calcular matrices de confusión
    cm_train = confusion_matrix(y_train, y_train_pred, labels=[0, 1])  # Asegurando el orden
    cm_test = confusion_matrix(y_test, y_test_pred, labels=[0, 1])

    cm_train_dict = {
    "type": "cm_matrix",
    "dataset": "train",
    "true_0": {
        "predicted_0": int(cm_train[0, 0]), 
        "predicted_1": int(cm_train[0, 1]),
        "support": int(sum(cm_train[0]))  # Total soporte para la clase 0
    },
    "true_1": {
        "predicted_0": int(cm_train[1, 0]), 
        "predicted_1": int(cm_train[1, 1]),
        "support": int(sum(cm_train[1]))  # Total soporte para la clase 1
    },
}


    cm_test_dict = {
        "type": "cm_matrix",
        "dataset": "test",
        "true_0": {"predicted_0": int(cm_test[0, 0]), "predicted_1": int(cm_test[0, 1])},
        "true_1": {"predicted_0": int(cm_test[1, 0]), "predicted_1": int(cm_test[1, 1])},
    }

    # Guardar métricas y matrices en un archivo JSON
    output_directory = os.path.abspath("../files/output")
    output_file = "metrics.json"
    output_path = os.path.join(output_directory, output_file)

    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)

    try:
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(json.dumps(metrics_train) + "\n")
            file.write(json.dumps(metrics_test) + "\n")
            file.write(json.dumps(cm_train_dict) + "\n")
            file.write(json.dumps(cm_test_dict) + "\n")
        print(f"Métricas y matrices de confusión guardadas correctamente en: {output_path}")
    except Exception as e:
        print(f"Error al guardar las métricas y matrices de confusión: {e}")
    
    # Imprimir el reporte de clasificación para verificación
    print("Reporte de clasificación (entrenamiento):")
    print(classification_report(y_train, y_train_pred, target_names=['Clase 0', 'Clase 1']))
    print("\nReporte de clasificación (prueba):")
    print(classification_report(y_test, y_test_pred, target_names=['Clase 0', 'Clase 1']))


# Ejemplo de uso
estimator = load_estimator()
save_metrics_and_confusion_matrices(estimator, X_train, y_train, X_test, y_test)


Métricas y matrices de confusión guardadas correctamente en: d:\ever\PhD\Predictiva\2024-2-LAB-01-prediccion-del-default-usando-rf-Ever708ch\files\output\metrics.json
Reporte de clasificación (entrenamiento):
              precision    recall  f1-score   support

     Clase 0       0.90      0.99      0.94     16228
     Clase 1       0.94      0.61      0.74      4725

    accuracy                           0.90     20953
   macro avg       0.92      0.80      0.84     20953
weighted avg       0.91      0.90      0.90     20953


Reporte de clasificación (prueba):
              precision    recall  f1-score   support

     Clase 0       0.85      0.94      0.90      7073
     Clase 1       0.65      0.39      0.49      1906

    accuracy                           0.83      8979
   macro avg       0.75      0.67      0.69      8979
weighted avg       0.81      0.83      0.81      8979

